# Truy cập trực tiếp cột first từ DataFrame thay vì dùng get()



Đúng rồi, hãy xem cách code đang xử lý cột `first`:

```python
# Cách cũ - có vấn đề
first_input = row.get('first', '')  # Dùng get() sẽ return '' nếu không tìm thấy cột 'first'
```

Sửa lại cách lấy dữ liệu từ cột `first`:

```python:deploy/backend/scripts/generate_story.py
def process_initial_data():
    try:
        logger.info("Starting initial data processing")
        
        # Read input Excel file
        df_input = pd.read_excel(UPLOAD_FOLDER / 'data.xlsx')
        logger.info(f"Columns in input file: {df_input.columns.tolist()}")  # Log để kiểm tra tên cột
        
        output_rows = []
        processed_count = 0

        for index, row in df_input.iterrows():
            order = row['order']
            prompt = row['prompt']
            # Lấy giá trị first trực tiếp từ DataFrame
            first_input = row['first'] if 'first' in df_input.columns else None
            
            logger.debug(f"Row {index} data:")
            logger.debug(f"- Order: {order}")
            logger.debug(f"- First input raw value: {first_input}")
            
            # Xử lý first_input
            if pd.isna(first_input):
                logger.warning(f"First input is NaN for order {order}")
                first_input = ""
            else:
                first_input = str(first_input).strip()
                
            logger.debug(f"- First input after processing: {first_input}")
            
            # Kiểm tra prompt
            if pd.isna(prompt) or str(prompt).strip() == '':
                logger.warning(f"Empty prompt for order {order}, skipping")
                continue
            
            # Process với first_input đã được xử lý
            responses, response_times, _ = process_conversation(order, prompt, [first_input])
            
            if responses:
                output_row = {
                    'order': order,
                    'User Input': first_input,
                    'json': responses[0],
                    'Response Time': response_times[0]
                }
                output_rows.append(output_row)
                processed_count += 1
                logger.info(f"Successfully processed row {index} with order {order}")
            
        # Create output DataFrame
        df_output = pd.DataFrame(output_rows)
        
        # Log processing results
        logger.info(f"Processing complete:")
        logger.info(f"- Total input rows: {len(df_input)}")
        logger.info(f"- Successfully processed: {processed_count}")
        logger.info(f"- Output shape: {df_output.shape}")
        
        # Save output
        output_path = OUTPUT_FOLDER / 'output_draft.xlsx'
        df_output.to_excel(output_path, index=False)
        
        return output_path
        
    except Exception as e:
        logger.error(f"Error in process_initial_data: {str(e)}", exc_info=True)
        raise
```

Những thay đổi chính:
1. Truy cập trực tiếp cột `first` từ DataFrame thay vì dùng `get()`
2. Kiểm tra sự tồn tại của cột `first`
3. Xử lý giá trị NaN một cách rõ ràng
4. Thêm logging chi tiết để dễ debug

Bạn thử chạy lại với những thay đổi này xem sao? Log sẽ cho bạn thấy chính xác giá trị của cột `first` trước và sau khi xử lý.
